In [1]:
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import feature_handler as handler

## Import data
- name : stock list
- date : date we collected
- clear : stock data we crawl dowm

In [2]:
Stock_list = pd.read_csv('name.csv')['name']
date = pd.read_csv('date.csv')['date']
Stock = pd.read_csv('clear.csv')
Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
Stock = Stock.reset_index(drop=True)

## Build all or Update
- update flag for solving time rebuild all
- Stock_PRE : previous data for model training

In [3]:
#update = False
update = True
if update :
    Stock_PRE = pd.read_csv('model.csv')
    finall_date = max(Stock_PRE.date)
    Stock_PRE

## building feature for model

In [4]:
list_1 = []
list_2 = []
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    handler.前高(s,list_1)
    handler.波段前高(s,list_2)
Stock['年前高'] = list_1
Stock['季前高'] = list_2

In [5]:
if update :
    start_date = date[date[date.values == finall_date ].index + 80].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [6]:
handler.漲跌(Stock)

In [7]:
handler.紅棒(Stock ,"實紅棒","紅棒天線")

In [8]:
handler.黑棒(Stock , "實黑棒","黑棒天線")

In [9]:
handler.避雷針(Stock , "上避雷針","下避雷針")

In [10]:
handler.連續(Stock , "連漲跌")

In [11]:
handler.季均漲跌天數(Stock , 60)

In [12]:
feature = 'MA60'
label = '收盤價'
test_ = []
day = int(''.join([x for x in feature if x.isdigit()]))
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    handler.增加技術指標MA(s,day,label,test_)        
Stock[feature] = test_

In [13]:
if update :
    start_date = date[date[date.values == finall_date ].index + 30].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [14]:
# adding MA feature
features =['MA5', 'MA10', 'MA20', 'buy5', 'buy10','外5', '外10', '投5','投10', '自5', '三大5', '三大10' ]
labels = ['收盤價', '收盤價','收盤價', '成交股數', '成交股數','外資進出','外資進出', '投信進出', '投信進出', '自營進出', '三大', '三大'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in Stock_list :
        s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
        handler.增加技術指標MA(s,day,labels[l],test_)        
    Stock[features[l]] = test_

In [15]:
handler.day_Volatility(Stock, '日振幅')

In [16]:
handler.Implied_Volatility(Stock,'波動率',20)

In [17]:
if update :
    start_date = date[date[date.values == finall_date ].index + 10].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [18]:
handler.MA交叉型態(Stock)

-1


In [19]:
handler.KD(Stock)

In [20]:
handler.KD交叉型態(Stock)

In [21]:
handler.KD交叉型態2(Stock)

In [22]:
handler.增加技術指標RSI(Stock)

division by zero zero 0
division by zero zero 1
division by zero zero 2
division by zero zero 3
division by zero zero 4


In [23]:
feature = ['MA5 slope','MA10 slope','MA60 slope','ratio slope']
label = ['MA5','MA10','MA60','ratio' ]
for i in range(len(label)) :
    handler.power_Slope(Stock , feature[i] , label[i])

In [24]:
feature = ['margin slope','short slope','三大 slope','外資5 slope','投信5 slope','自營 slope','buy5 slope','buy10 slope','MA5 acc']
label = ['融資餘','融卷餘','三大5','外5','投5', '自5', 'buy5','buy10','MA5 slope']
for i in range(len(label)) :
    handler.Slope(Stock , feature[i] , label[i])

C:\Users\kevin\Desktop\Stock\feature_handler.py:306: RuntimeWarning: divide by zero encountered in double_scalars
  K = (df[label][i]-df[label][i-1])/(df[label][i-1]+0.001)*100
C:\Users\kevin\Desktop\Stock\feature_handler.py:306: RuntimeWarning: invalid value encountered in double_scalars
  K = (df[label][i]-df[label][i-1])/(df[label][i-1]+0.001)*100


In [25]:
handler.RSI鈍化(Stock)

In [26]:
feature = ['外5金','投5金','自5金','融資金','融卷金']
label1 = ['收盤價','收盤價','收盤價','收盤價','收盤價']
label2 = ['外5','投5','自5','融資買','融卷賣']
for i in range(len(feature)):
    handler.intercept(Stock , feature[i] , label1[i] ,label2[i])

In [27]:
feature = ['D5','D60','D三大','D_upper_day']
label1 = ['收盤價','收盤價','三大5','收盤價']
label2 = ['MA5','MA60','三大10','最高價']
for i in range(len(feature)):
    handler.distance(Stock , feature[i] , label1[i] ,label2[i])

In [28]:
feature = ['UD5','UD60','UD前高壓力','UD波段壓立']
label1 = ['收盤價','收盤價','收盤價','收盤價']
label2 = ['MA5','MA60','年前高','季前高']
for i in range(len(feature)):
    handler.Updown_state(Stock , feature[i] , label1[i] ,label2[i])

In [29]:
handler.均線糾結(Stock)

C:\Users\kevin\Desktop\Stock\feature_handler.py:505: RuntimeWarning: invalid value encountered in double_scalars
  d = min(MA5,MA10,MA20) / max(MA5,MA10,MA20)
C:\Users\kevin\Desktop\Stock\feature_handler.py:506: RuntimeWarning: invalid value encountered in double_scalars
  d2 = min(MA5,MA10,MA20,MA60) / max(MA5,MA10,MA20,MA60)


In [30]:
handler.三陽開泰 (Stock)

In [31]:
handler.突破前高(Stock)

In [32]:
handler.突破整理(Stock)

max() arg is an empty sequence 0
max() arg is an empty sequence 1
max() arg is an empty sequence 2
max() arg is an empty sequence 3
max() arg is an empty sequence 4
max() arg is an empty sequence 5
max() arg is an empty sequence 6
max() arg is an empty sequence 7
max() arg is an empty sequence 8
max() arg is an empty sequence 9


In [33]:
feature = ['均買量','R_散戶']
label1 = ['成交金額','三大 slope']
label2 = ['成交筆數','margin slope']
for i in range(len(feature)):
    handler.Ratio(Stock , feature[i] , label1[i] ,label2[i])

In [34]:
handler.超漲跌(Stock)

In [35]:
days = [5,3,1]
uppers = [0.1 , 0.05 ,0.03 ]
for i in range(len(days)):
    handler.shift_price(Stock, days[i] , uppers[i])

In [36]:
if update :
    Stock_new = Stock[Stock.date > finall_date].reset_index(drop = True)
    Stock_new
    Stock = Stock_PRE.append(Stock_new)
    Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
    Stock = Stock.reset_index(drop=True)

In [37]:
Stock_index = pd.read_csv('Market_index.csv')
Stock_index.sort_values(by=['Index','date'] , ascending=True, axis =0 , inplace=True)
index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
Stock_index = Stock_index.reset_index(drop=True)


features =['MA5', 'MA20' ]
labels = ['收盤', '收盤'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in index_set:
        s = Stock_index[Stock_index.Index == i].reset_index(drop=True)
        handler.增加技術指標MA(s,day,labels[l],test_)
        
    Stock_index[features[l]] = test_

features =['MA5 slop', 'MA20 slop' ]
labels = ['MA5', 'MA20'] 
for l in range(len(labels)):
    handler.Slope (Stock_index , features[l] , labels[l] )

features =['乖離5', '乖離20' ]
label1 = ['收盤','收盤']
label2 = ['MA5','MA20']
for i in range(len(labels)):
    handler.distance (Stock_index , features[i] , label1[i] ,label2[i] )
    
Stock_index

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
0,未含電子指數,15054.77,65.44,0.44,20190429,0.000,0.0000,0.0000,0.0000,1.00,1.00
1,未含電子指數,15107.62,52.85,0.35,20190430,0.000,0.0000,0.0000,0.0000,1.00,1.00
2,未含電子指數,15197.27,89.65,0.59,20190502,0.000,0.0000,0.0000,0.0000,1.00,1.00
3,未含電子指數,15194.64,2.63,-0.02,20190503,0.000,0.0000,0.0000,0.0000,1.00,1.00
4,未含電子指數,15004.13,190.51,-1.25,20190506,0.000,0.0000,0.0000,0.0000,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...
1535,電子類兩倍槓桿指數,42705.36,894.33,2.14,20210415,50761.456,42898.8370,0.0607,0.3182,-0.19,-0.00
1536,電子類兩倍槓桿指數,42617.60,87.76,-0.21,20210416,50653.656,43009.7775,-0.2124,0.2586,-0.19,-0.01
1537,電子類兩倍槓桿指數,42056.33,561.27,-1.32,20210419,50542.926,43118.5295,-0.2186,0.2529,-0.20,-0.03
1538,電子類兩倍槓桿指數,42467.73,411.40,0.98,20210420,50657.364,43224.8265,0.2264,0.2465,-0.19,-0.02


In [38]:
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
Market

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
483,發行量加權股價指數,10145.28,80.50,0.80,20190218,18640.374,16440.3285,0.0051,0.2204,-0.84,-0.62
484,發行量加權股價指數,10152.26,6.98,0.07,20190219,18717.678,16469.7485,0.4147,0.1790,-0.84,-0.62
485,發行量加權股價指數,10272.46,120.20,1.18,20190220,18962.474,16514.6655,1.3078,0.2727,-0.85,-0.61
486,發行量加權股價指數,10319.53,47.07,0.46,20190221,19165.358,16558.0260,1.0699,0.2626,-0.86,-0.60
487,發行量加權股價指數,10322.92,3.39,0.03,20190222,19281.624,16600.0600,0.6066,0.2539,-0.87,-0.61
...,...,...,...,...,...,...,...,...,...,...,...
1006,發行量加權股價指數,17076.73,210.76,1.25,20210415,21060.418,17805.8865,0.6782,0.3840,-0.23,-0.04
1007,發行量加權股價指數,17158.81,82.08,0.48,20210416,21238.088,17886.5420,0.8436,0.4530,-0.24,-0.04
1008,發行量加權股價指數,17263.28,104.47,0.61,20210419,21521.742,17997.6265,1.3356,0.6211,-0.25,-0.04
1009,發行量加權股價指數,17323.87,60.59,0.35,20210420,21751.668,18108.1780,1.0683,0.6143,-0.26,-0.05


In [41]:
date

0      20210421
1      20210420
2      20210419
3      20210416
4      20210415
         ...   
525    20190221
526    20190220
527    20190219
528    20190218
529    20190215
Name: date, Length: 530, dtype: int64

In [39]:
Stock.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比',
       ...
       'R_散戶', 'pre5d0.1', 'pre5p0.1', 'pre3d0.05', 'pre3p0.05', 'pre1d0.03',
       'pre1p0.03', '均漲天數', '均跌天數', '超漲跌'],
      dtype='object', length=104)

list_ma5 = []
list_ma20 = []
list_market = []
list_5D = []
list_20D = []
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
for i in range(len(Stock)):
    day = Stock.date[i].squeeze()
    market = Market[(Market.date == int(day))]
    list_ma5.append(market['MA5 slop'].squeeze())
    list_ma20.append(market['MA20 slop'].squeeze())
    list_market.append(market.漲跌.squeeze())
    list_5D.append(market.乖離5.squeeze())
    list_20D.append(market.乖離20.squeeze())
    

Stock['大盤漲跌'] = list_market
Stock['大盤5MA'] = list_ma5
Stock['大盤20MA'] = list_ma20
Stock['大盤5乖離'] = list_5D
Stock['大盤20乖離'] = list_20D

In [40]:
Stock.to_csv('model.csv', index=False)